In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from math import sqrt

In [3]:
movies_cols = ['MovieID','Title','Genres']
ratings_cols = ['UserID','MovieID','Rating','Timestamp']
users_cols = ['UserID','Gender','Age','Occupation','Zip-code']

In [4]:
# DataSet 1
movies = pd.read_csv('/content/drive/MyDrive/DataSets/Movie-rating/movies.dat', sep = '::',header = None, names = movies_cols, encoding = 'latin 1')

# DataSet 2
ratings = pd.read_csv('/content/drive/MyDrive/DataSets/Movie-rating/ratings.dat',sep = '::', header = None, names = ratings_cols)

# DataSet 3
users = pd.read_csv('/content/drive/MyDrive/DataSets/Movie-rating/users.dat',sep = '::', header = None, names = users_cols)

<ipython-input-4-f697a4e4e1cd>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_csv('/content/drive/MyDrive/DataSets/Movie-rating/movies.dat', sep = '::',header = None, names = movies_cols, encoding = 'latin 1')
<ipython-input-4-f697a4e4e1cd>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv('/content/drive/MyDrive/DataSets/Movie-rating/ratings.dat',sep = '::', header = None, names = ratings_cols)
<ipython-input-4-f697a4e4e1cd>:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and diff

In [34]:
movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [35]:
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [36]:
users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


## Creating a [Master_Data] dataset with the columns:

MovieID Title UserID Age Gender Occupation Rating

In [37]:
index_on_movieid = pd.merge(movies, ratings, on = 'MovieID').reset_index(drop = True)
index_on_movieid.head()

,MovieID,Title,Genres,UserID,Rating,Timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268
1,1,Toy Story (1995),Animation|Children's|Comedy,6,4,978237008
2,1,Toy Story (1995),Animation|Children's|Comedy,8,4,978233496
3,1,Toy Story (1995),Animation|Children's|Comedy,9,5,978225952
4,1,Toy Story (1995),Animation|Children's|Comedy,10,5,978226474


In [38]:
master_data = pd.merge(index_on_movieid, users, on = 'UserID').reset_index(drop = True)
master_data.head()

,MovieID,Title,Genres,UserID,Rating,Timestamp,Gender,Age,Occupation,Zip-code
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268,F,1,10,48067
1,48,Pocahontas (1995),Animation|Children's|Musical|Romance,1,5,978824351,F,1,10,48067
2,150,Apollo 13 (1995),Drama,1,5,978301777,F,1,10,48067
3,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,1,4,978300760,F,1,10,48067
4,527,Schindler's List (1993),Drama|War,1,5,978824195,F,1,10,48067


In [39]:
master_data.to_csv(r'/content/drive/MyDrive/DataSets/Movie-rating/master_data.csv',index = False)

In [40]:
master_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   MovieID     1000209 non-null  int64 
 1   Title       1000209 non-null  object
 2   Genres      1000209 non-null  object
 3   UserID      1000209 non-null  int64 
 4   Rating      1000209 non-null  int64 
 5   Timestamp   1000209 non-null  int64 
 6   Gender      1000209 non-null  object
 7   Age         1000209 non-null  int64 
 8   Occupation  1000209 non-null  int64 
 9   Zip-code    1000209 non-null  object
dtypes: int64(6), object(4)
memory usage: 76.3+ MB


In [72]:
df = pd.read_csv('/content/drive/MyDrive/DataSets/Movie-rating/master_data.csv')
df.head()

,MovieID,Title,Genres,UserID,Rating,Timestamp,Gender,Age,Occupation,Zip-code
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268,F,1,10,48067
1,48,Pocahontas (1995),Animation|Children's|Musical|Romance,1,5,978824351,F,1,10,48067
2,150,Apollo 13 (1995),Drama,1,5,978301777,F,1,10,48067
3,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,1,4,978300760,F,1,10,48067
4,527,Schindler's List (1993),Drama|War,1,5,978824195,F,1,10,48067


In [73]:
df = df.drop(['Genres','Timestamp','Zip-code','Occupation'],axis = 1)   #dont remove if Content-based Collaborative Filtering
df.head()

,MovieID,Title,UserID,Rating,Gender,Age
0,1,Toy Story (1995),1,5,F,1
1,48,Pocahontas (1995),1,5,F,1
2,150,Apollo 13 (1995),1,5,F,1
3,260,Star Wars: Episode IV - A New Hope (1977),1,4,F,1
4,527,Schindler's List (1993),1,5,F,1


In [74]:
df['Gender'] = df['Gender'].replace({'M':1,'F':0})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 6 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   MovieID  1000209 non-null  int64 
 1   Title    1000209 non-null  object
 2   UserID   1000209 non-null  int64 
 3   Rating   1000209 non-null  int64 
 4   Gender   1000209 non-null  int64 
 5   Age      1000209 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 45.8+ MB


I built a Recommendation system that predicts how a user would rate a movie based on their historical ratings and ratings of similar users. Hence MovieId, UserId and Ratings are taken into consideration. Any advanced algorithm would need as much as information like "Age" and "Gender" of user.

In [75]:
user_ids = df['UserID'].unique()
movie_ids = df['MovieID'].unique()

num_users = len(user_ids)
num_movies = len(movie_ids)

user_id_to_index = {user_id: index for index, user_id in enumerate(user_ids)}
movie_id_to_index = {movie_id: index for index, movie_id in enumerate(movie_ids)}

df['UserID'] = df['UserID'].map(user_id_to_index)
df['MovieID'] = df['MovieID'].map(movie_id_to_index)
df

,MovieID,Title,UserID,Rating,Gender,Age
0,0,Toy Story (1995),0,5,0,1
1,1,Pocahontas (1995),0,5,0,1
2,2,Apollo 13 (1995),0,5,0,1
3,3,Star Wars: Episode IV - A New Hope (1977),0,4,0,1
4,4,Schindler's List (1993),0,5,0,1
...,...,...,...,...,...,...
1000204,288,Rules of Engagement (2000),6039,4,1,25
1000205,1700,American Psycho (2000),6039,2,1,25
1000206,102,Keeping the Faith (2000),6039,5,1,25
1000207,1480,U-571 (2000),6039,3,1,25


In [80]:
# Create data splits
X = df[['MovieID','UserID']]
y = df['Rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [81]:
from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Concatenate

# Define embedding dimensions
embedding_dim = 32

# Create user and movie embedding layers
user_input = Input(shape=(1,))
user_embedding = Embedding(input_dim=num_users, output_dim=embedding_dim)(user_input)
user_embedding = Flatten()(user_embedding)

movie_input = Input(shape=(1,))
movie_embedding = Embedding(input_dim=num_movies, output_dim=embedding_dim)(movie_input)
movie_embedding = Flatten()(movie_embedding)

# Concatenate user and movie embeddings
concat = Concatenate()([user_embedding, movie_embedding])

In [82]:
# Create the collaborative filtering model
dense_1 = tf.keras.layers.Dense(128, activation='relu')(concat)
dense_2 = tf.keras.layers.Dense(64, activation='relu')(dense_1)
output = tf.keras.layers.Dense(1)(dense_2)

model = Model(inputs=[user_input, movie_input], outputs=output)

In [83]:
# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

# Train the model
model.fit([X_train['UserID'], X_train['MovieID']], y_train, epochs=5, batch_size=64, verbose=1)

# Evaluate the model
loss, mae = model.evaluate([X_test['UserID'], X_test['MovieID']], y_test)
print(f'Test MAE: {mae}')

Epoch 1/5
12503/12503 [==============================] - 86s 7ms/step - loss: 0.8929 - mae: 0.7431
Epoch 2/5
12503/12503 [==============================] - 80s 6ms/step - loss: 0.7927 - mae: 0.7023
Epoch 3/5
12503/12503 [==============================] - 82s 7ms/step - loss: 0.7612 - mae: 0.6867
Epoch 4/5
12503/12503 [==============================] - 80s 6ms/step - loss: 0.7349 - mae: 0.6732
Epoch 5/5
6252/6252 [==============================] - 11s 2ms/step - loss: 0.7606 - mae: 0.6844
Test MAE: 0.6844174265861511


In [93]:
# User input for prediction
user_id_input = user_id_to_index[20]  # Replace with the desired user ID index
movie_id_input = movie_id_to_index[1799]  # Replace with the desired movie ID index

# Predict the rating
predicted_rating = model.predict([np.array([user_id_input]), np.array([movie_id_input])])
#result = df.query("MovieID == @movie_id_input")['Title']
resultUser = df[['Gender','Age']][df.UserID == user_id_input]
resultMovie = df[['Title']][df.MovieID == movie_id_input]
print(f"Predicted rating for user {resultUser.values[0]} and movie {resultMovie.values[0]}: {predicted_rating[0][0]}")

1/1 [==============================] - 0s 23ms/step
Predicted rating for user [ 1 25] and movie ['Suicide Kings (1997)']: 3.9635634422302246
